In [ ]:
import requests
from lxml import html

import unicodecsv as csv
import pandas as pd
import numpy as np

import time
import random
import re
import json

from IPython.display import display, Markdown, HTML
import IPython.display as dsp

pd.set_option('display.width', None)
pd.set_option('display.max_rows', 120)
pd.set_option('display.max_info_rows', 120)
pd.set_option('display.max_info_columns', 120)
pd.options.display.max_colwidth = 200  # pd.set_option('display.max_colwidth', 200)
pd.set_option('display.float_format', lambda fp: f'{fp:,.3f}')


import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.preprocessing
import sklearn.impute
import sklearn.neighbors
import sklearn.cluster

import scipy.stats



In [ ]:
def write(name, data):
    with open(name, 'w') as f:
f.write(data)

In [141]:
def random_ua():
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.3',
        #'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
        #'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 Edg/130.0.0.0',
    ]
    selected_ua = random.choice(user_agents)
    print(f'Selected UA: {selected_ua}')
    return selected_ua

In [166]:
def countdown( min, max=None, buffer=None, prefix='Sleeping', postfix='...'):
    '''Print a countdown to screen.  Erase it when it is done.  
    Accepts either the time to countdown OR a min and max for randint for random sleeps.'''
    if max is None:
        max = min
    
    delay = random.randint(min, max)
    for i in range(delay):
        msg = f'{prefix} {delay-i:<2}s'
        msg = f'{msg:<14}s / {delay:>2}s {postfix}'
        if buffer is None:
            print(msg, end='')
        else:
            buffer.update(Markdown(msg))

        time.sleep(1)
        if buffer is None:
            print(f'\r                      ', end='\r')
        #else:
            #buffer.update(Markdown(f' '))
    if buffer is None:
        print(LINE_UP, LINE_CLEAR, end='')
    else:
        buffer.update(Markdown('Sleeping complete ...'))
    

In [ ]:
def write_csv(data, target, fieldnames=None):
    # Save data to csv

    with open(f'properties-{target}.csv', 'wb') as csvfile:
if fieldnames is None:
            fieldnames = list(data[0].keys())
        print(f'Fieldnames: {fieldnames}')
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

In [174]:
def pr(msg, buffer=None, end='\n', auto_format=True):
    #format = dict(top='## {}', mid='{}', bot='{}')  # buffers are IPython.display.display objects with a 'name' attribute added

    if not buffer:
        print(msg, end=end)
    
    else:
       #print('PR: msg type: ', type(msg))
       if isinstance(msg, str):
        if auto_format:
            try:
              msg = buffer.format.format(msg)
            except Exception as e:
              #print('In pr(): buffer formatting failed')
              pass
        buffer.update(Markdown(msg))
       
       else:
          #print('PR: msg type: ',type(msg), 'auto_format: ', auto_format)
          buffer.update(msg)


In [163]:
isinstance('', str)

True

In [132]:
def get_display_handles():
    dh_top = display('...', display_id=True)
    dh_mid = display('...', display_id=True)
    dh_bot = display('...', display_id=True)

    dh_top.name = 'top'
    dh_mid.name = 'mid'
    dh_bot.name = 'bot'

    dh_top.format = '#### {}'
    dh_bot.format = '{}'
    dh_mid.format = '{}'

    return (dh_top, dh_mid, dh_bot)

In [138]:
def RF_get_df(parser, index=None):
    '''Extract the entire page of search results from the buried JSON found in the HTML.  parse: lxml.html object'''
    df = None
    for path in parser.xpath('//div[@id="content"]/script'):
        try:
            match = re.findall(r'reactServerState\.InitialContext = (\{.*\})', path.text_content())[0]  # should load big honking list of matches
            
            try:
                js = json.loads(match)['ReactServerAgent.cache']['dataCache']
            except:
                next
            
            key = None
            for k in js.keys():
                if '/gis?' in k:
                    key = k
                    break
            str = js[k]['res']['text'][4:]
            df = pd.json_normalize(json.loads(str)['payload']['homes'])
        
        except:
            next
    if index is not None:
        df = df.set_index(index)
    return df

In [130]:
def RF_next_url_suffix(parser):
    url = parser.xpath('//a[contains(@class, "__page--selected")]/following-sibling::a/@href')
    if len(url) >= 1:
        url = parser.xpath('//a[contains(@class, "__page--selected")]/following-sibling::a/@href')[0]
    else:
        url = None
    return url

In [176]:
def RF_fetch_data(target=None, filter=''):
    base_url = 'https://www.redfin.com{}'
    if target is None:
        target = '/county/2362/PA/Allegheny-County/'
        #url = 'https://www.redfin.com/county/2362/PA/Allegheny-County/'
    else:
        if target[0] != '/':
            target = '/' + target
        if target[-1] != '/':
            target += '/'
        if filter != '':
            if filter[0] == '/':
                filter = filter[1:]
            if filter[-1] != '/':
                filter += '/'
    url = base_url.format(target+filter)

    headers = {
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "accept-language": "en-US,en;q=0.9",
        "cache-control": "max-age=0",
        "priority": "u=0, i",
        "sec-ch-ua": "\"Chromium\";v=\"130\", \"Microsoft Edge\";v=\"130\", \"Not?A_Brand\";v=\"99\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "same-origin",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 Edg/130.0.0.0',
    }
    s = requests.Session()
    s.headers.update(headers)

    df = None
    dh_top, dh_mid, dh_bot = get_display_handles()
    print(dh_top, dh_mid, dh_bot)
    #input()
    # Loop
    while url is not None:
        response = s.get(url=url)
        parser = html.fromstring(response.text)
        pr(f'**{response.status_code}**: {url}', buffer=dh_top)
        
        new_data = RF_get_df(parser, index='propertyId')
        if df is None:
            df = new_data
        else:
            df = pd.concat((df, new_data))
            pr(f'Page {url[-1]}: Added {df.shape[0]:>5} new rows.', buffer=dh_bot)

        last_url = url
        next_suffix = RF_next_url_suffix(parser)
        url = base_url.format(next_suffix) if next_suffix is not None else None
        if url is None or last_url == url:
            pr(f'**BREAKING**: No more URLs. Last URL: {last_url}', buffer=dh_mid)
            break

        countdown(min=3, max=7, buffer=dh_mid, postfix=f'Next URL: {url}')   # Sleep
        pr(f'Looping from beginning.  Next URL: {url}', buffer=dh_mid)
    
    return df

In [173]:
df = RF_fetch_data(filter='filter/include=sold-3yr/')
df

#### **200**: https://www.redfin.com/county/2362/PA/Allegheny-County/page-8

**BREAKING**: No more URLs. Last URL: https://www.redfin.com/county/2362/PA/Allegheny-County/page-8

Page 8: Added  2800 new rows.

<DisplayHandle display_id=4496286afa3deaef4f061b901a7ae475> <DisplayHandle display_id=928e520c273a59a209343f774ae8f167> <DisplayHandle display_id=c1d44a9d81c9abd917e07767eb4a9943>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>
PR: msg type:  <class 'str'>


,showMlsId,mlsStatus,showDatasourceLogo,hideSalePrice,isHoaFrequencyKnown,beds,baths,fullBaths,city,state,...,listingAgent.name,listingAgent.redfinAgentId,partialBaths,stories,openHouseStart,openHouseEnd,openHouseStartFormatted,openHouseEventName,listingBroker.phone,buildingId
propertyId,,,,,,,,,,,,,,,,,,,,,
74748918,False,Active,True,False,True,2,2.000,2.000,Shadyside,PA,...,Edward Nadolny,"16,415.000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74813533,False,Active,True,False,True,3,2.500,2.000,Adams Twp,PA,...,NaN,NaN,1.000,2.000,"1,731,772,800,000.000","1,731,780,000,000.000","Nov 16, 11:00AM",Open House - 11:00 - 1:00 PM,NaN,NaN
73499663,False,Active,True,False,True,4,3.000,3.000,Wilkinsburg,PA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73513439,False,Active,True,False,True,4,2.500,2.000,Pine Twp - Nal,PA,...,NaN,NaN,1.000,2.000,"1,731,772,800,000.000","1,731,780,000,000.000","Nov 16, 11:00AM",Open House - 11:00 - 1:00 PM,NaN,NaN
73568231,False,Active,True,False,True,3,2.500,2.000,Hampton,PA,...,NaN,NaN,1.000,1.000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112774188,False,Active,True,False,True,0,NaN,NaN,Ross Twp,PA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193441060,False,Active,True,False,True,0,NaN,NaN,East Liberty,PA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74682029,False,Active,True,False,True,0,NaN,NaN,Hazelwood,PA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
df_sale = RF_fetch_data()
df_sale

#### **200**: https://www.redfin.com/county/2362/PA/Allegheny-County/page-8

**BREAKING**: No more URLs. Last URL: https://www.redfin.com/county/2362/PA/Allegheny-County/page-8

Page 8: Added  2800 new rows.

<DisplayHandle display_id=e9585eae300d1e23c4866bebfe2774d6> <DisplayHandle display_id=3acdf816cea74a3aab0a9b6e611f255a> <DisplayHandle display_id=a748466099abbef05510157bda4e4db0>


,showMlsId,mlsStatus,showDatasourceLogo,hideSalePrice,isHoaFrequencyKnown,beds,baths,fullBaths,city,state,...,listingAgent.name,listingAgent.redfinAgentId,partialBaths,stories,openHouseStart,openHouseEnd,openHouseStartFormatted,openHouseEventName,listingBroker.phone,buildingId
propertyId,,,,,,,,,,,,,,,,,,,,,
74748918,False,Active,True,False,True,2,2.000,2.000,Shadyside,PA,...,Edward Nadolny,"16,415.000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74777740,False,Active,True,False,True,3,2.500,2.000,Richland,PA,...,NaN,NaN,1.000,2.000,NaN,NaN,NaN,NaN,NaN,NaN
74813533,False,Active,True,False,True,3,2.500,2.000,Adams Twp,PA,...,NaN,NaN,1.000,2.000,"1,731,772,800,000.000","1,731,780,000,000.000","Nov 16, 11:00AM",Open House - 11:00 - 1:00 PM,NaN,NaN
73499663,False,Active,True,False,True,4,3.000,3.000,Wilkinsburg,PA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73513439,False,Active,True,False,True,4,2.500,2.000,Pine Twp - Nal,PA,...,NaN,NaN,1.000,2.000,"1,731,772,800,000.000","1,731,780,000,000.000","Nov 16, 11:00AM",Open House - 11:00 - 1:00 PM,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112774188,False,Active,True,False,True,0,NaN,NaN,Ross Twp,PA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193441060,False,Active,True,False,True,0,NaN,NaN,East Liberty,PA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74682029,False,Active,True,False,True,0,NaN,NaN,Hazelwood,PA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_sale.soldDate = pd.to_datetime(df_sale.soldDate, unit='ms')
df.soldDate = pd.to_datetime(df.soldDate, unit='ms')

In [243]:
id_cols = ['showMlsId', 'dataSourceId', 'businessMarketId']
flag_cols = ['showDatasourceLogo', 'hideSalePrice', 'isHoaFrequencyKnown', 'hasInsight', 'isHot', 'hasVirtualTour', 'hasVideoTour', 'has3DTour', 'isRedfin', 'isActiveKeyListing', 'isNewConstruction', 'isShortlisted', 'isViewedListing', ]

home_features_cols = ['beds', 'baths', 'fullBaths','propertyType', 'uiPropertyType', 'keyFacts', 'listingRemarks', 'price.value']
location_cols = ['city', 'state', 'zip', 'countryCode', 'showAddressOnMap', 'soldDate', 'searchStatus', 'marketId', 'timeZone', ]
listing_cols = ['listingType', 'listingId', 'mlsStatus',]
photo_cols = ['primaryPhotoDisplayLevel', 'additionalPhotosInfo', ]
url_cols = ['scanUrl', 'posterFrameUrl', 'url', ]
other_cols = ['sashes', 'remarksAccessLevel',
       'mlsId.label',
       'mlsId.value', 'price.level', 'hoa.value', 'hoa.level',
       'sqFt.value', 'sqFt.level', 'pricePerSqFt.value', 'pricePerSqFt.level',
       'lotSize.value', 'lotSize.level', 'location.value', 'location.level',
       'latLong.value.latitude', 'latLong.value.longitude', 'latLong.level',
       'streetLine.value', 'streetLine.level', 'unitNumber.value',
       'unitNumber.level', 'postalCode.value', 'postalCode.level',
       'yearBuilt.value', 'yearBuilt.level', 'dom.value', 'dom.level',
       'timeOnRedfin.value', 'timeOnRedfin.level',
       'originalTimeOnRedfin.value', 'originalTimeOnRedfin.level',
       'photos.value', 'photos.level', 'alternatePhotosInfo.mediaListType',
       'alternatePhotosInfo.mediaListIndex', 'alternatePhotosInfo.groupCode',
       'alternatePhotosInfo.positionSpec', 'alternatePhotosInfo.type',
       'listingBroker.name', 'listingBroker.isRedfin',
       'sellingBroker.isRedfin', 'listingAgent.name',
       'listingAgent.redfinAgentId', 'partialBaths', 'stories',
       'openHouseStart', 'openHouseEnd', 'openHouseStartFormatted',
       'openHouseEventName', 'listingBroker.phone', 'buildingId'
]
df[home_features_cols].listingRemarks.iat[0]

"This charming 2-bedroom, 2-full bath condo offers a perfect blend of comfort and convenience. Located close to Bakery Square, schools, as well as 71B and 71D, this unit features a spacious living room, a formal dining room, and an updated kitchen and bathrooms. Enjoy your morning coffee on the private balcony. The building was renovated in the 1950's, and the building's roof was renovated less than three years ago. You'll have the added benefits of assigned off-street parking, as well as storage and laundry facilities in the basement. The condo allows rentals. This year's condo dues and upcoming assessment are already paid off, providing savings! Condo assoc. includes water, gas, snow remov"

In [247]:

df[home_features_cols].groupby(['beds', 'baths'], ).mean(numeric_only=True)

fullBaths  propertyType  uiPropertyType   price.value
beds baths                                                        
1    1.000       1.000         3.000           2.000   174,966.667
     1.500       1.000         6.000           1.000   299,000.000
2    1.000       1.000         6.156           1.219   133,550.000
     1.500       1.000         6.125           1.500   246,250.000
     2.000       1.950         5.500           1.500   287,639.950
     2.500       2.000         8.000           2.500   294,950.000
     6.000       4.000         4.000           4.000   480,000.000
3    1.000       1.000         6.419           1.163   162,277.907
     1.500       1.000         6.023           1.093   244,227.907
     2.000       1.980         5.811           1.254   250,563.662
     2.500       2.000         8.307           1.835   397,188.630
     3.000       2.636         5.818           1.455   392,672.545
     3.500       3.000         8.750           2.250   789,037.500
     4.000       4.000         6.000           1.000   679,900.000
4    1.000       1.000         6.000           1.000   325,000.000
     1.500       1.000         6.000           1.000   250,074.875
     2.000       2.000         5.667           1.500   306,124.917
     2.500       2.000         6.000           1.000   511,547.059
     3.000       2.714         6.429           2.143   442,000.000
     3.500       3.000         6.636           1.182   812,990.000
     4.000       3.000         6.000           1.000   598,750.000
     4.500       4.000         6.000           1.000   797,450.000
5    1.000       1.000         6.000           1.000   180,000.000
     2.000       2.000         5.000           2.500   232,450.000
     2.500       2.000         6.000           1.000   536,950.000
     3.500       3.000         6.000           1.000   759,000.000
     4.500       4.000         6.000           1.000 1,087,449.500
     5.500       5.000         6.000           1.000 2,550,000.000
6    2.000       2.000         6.000           1.000    47,000.000
     3.000       3.000         6.000           1.000   150,000.000
     4.500       4.000         5.000           2.500 1,574,950.000
8    3.500       3.000         6.000           1.000   799,000.000
9    6.000       6.000         4.000           4.000   490,000.000
15   5.000       5.000         5.000           4.000   290,000.000
21   14.000     14.000         5.000           4.000   699,988.000

In [260]:
dfhf = df[home_features_cols]

dfhf.sort_index()

,beds,baths,fullBaths,propertyType,uiPropertyType,keyFacts,listingRemarks,price.value
propertyId,,,,,,,,
73492087,3,3.000,3.000,6,1,"[{'description': '2,848 sq ft lot', 'rank': 0}, {'description': 'Very walkable', 'rank': 1}]",A Versatile and Beautifully Maintained Gem located in Char Valley SD! ~ Step inside to solid hardwood floors and a semi-open concept family area! The spacious living room is incredibly flexible—us...,174999
73492087,3,3.000,3.000,6,1,"[{'description': '2,848 sq ft lot', 'rank': 0}, {'description': 'Very walkable', 'rank': 1}]",A Versatile and Beautifully Maintained Gem located in Char Valley SD! ~ Step inside to solid hardwood floors and a semi-open concept family area! The spacious living room is incredibly flexible—us...,174999
73492087,3,3.000,3.000,6,1,"[{'description': '2,848 sq ft lot', 'rank': 0}, {'description': 'Very walkable', 'rank': 1}]",A Versatile and Beautifully Maintained Gem located in Char Valley SD! ~ Step inside to solid hardwood floors and a semi-open concept family area! The spacious living room is incredibly flexible—us...,174999
73492087,3,3.000,3.000,6,1,"[{'description': '2,848 sq ft lot', 'rank': 0}, {'description': 'Very walkable', 'rank': 1}]",A Versatile and Beautifully Maintained Gem located in Char Valley SD! ~ Step inside to solid hardwood floors and a semi-open concept family area! The spacious living room is incredibly flexible—us...,174999
73492087,3,3.000,3.000,6,1,"[{'description': '2,848 sq ft lot', 'rank': 0}, {'description': 'Very walkable', 'rank': 1}]",A Versatile and Beautifully Maintained Gem located in Char Valley SD! ~ Step inside to solid hardwood floors and a semi-open concept family area! The spacious living room is incredibly flexible—us...,174999
...,...,...,...,...,...,...,...,...
193516844,4,1.500,1.000,6,1,"[{'description': '1.51 acre lot', 'rank': 0}, {'description': 'Somewhat walkable', 'rank': 1}]","Architecturally Rich and Classic homes from 1898, need some Tender Loving Care! The Main House and a Guest Home are on 1.51 Acres in the Heart of South Hills. Many Original and Charming features a...",400000
193516844,4,1.500,1.000,6,1,"[{'description': '1.51 acre lot', 'rank': 0}, {'description': 'Somewhat walkable', 'rank': 1}]","Architecturally Rich and Classic homes from 1898, need some Tender Loving Care! The Main House and a Guest Home are on 1.51 Acres in the Heart of South Hills. Many Original and Charming features a...",400000
193516844,4,1.500,1.000,6,1,"[{'description': '1.51 acre lot', 'rank': 0}, {'description': 'Somewhat walkable', 'rank': 1}]","Architecturally Rich and Classic homes from 1898, need some Tender Loving Care! The Main House and a Guest Home are on 1.51 Acres in the Heart of South Hills. Many Original and Charming features a...",400000


In [ ]:
a = json.loads('{"ReactServerAgent.cache":{"dataCache":{"\u002Fstingray\u002Fcorvair\u002Fv2\u002FpreresolveBouncerFlags":{"url":"\u002Fstingray\u002Fcorvair\u002Fv2\u002FpreresolveBouncerFlags","requestData":{"urlPath":"\u002Fstingray\u002Fcorvair\u002Fv2\u002FpreresolveBouncerFlags","method":"GET","queryParams":[],"postParams":null,"type":"json"},"requesters":2,"loaded":true,"res":{"_hasBody":true,"text":"{}&&{\"bouncerData\":{\"4496425\":\"1\",\"8537661\":\"1\",\"863218446\":\"1\",\"2428670\":\"1\",\"267639697\":\"Variant\",\"1203506167\":\"1\",\"967225084\":\"1\",\"52278200\":\"1\",\"1016995215\":\"1\",\"876931639\":\"1\",\"1455831115\":\"1\",\"1545207106\":\"1\",\"4\":\"1\",\"79525\":\"1\",\"9565569\":\"1\",\"1714458391\":\"VARIANT\",\"2068074617\":\"1\",\"988347912\":\"1\",\"381469020\":\"1\",\"621081811\":\"VARIANT\",\"324331314\":\"1\",\"128514355\":\"1\",\"1610565413\":\"1\",\"1584385382\":\"Variant\",\"1908214897\":\"1\",\"6347749\":\"1\",\"1210857133\":\"Variant\",\"902143405\":\"1\",\"4455910\":\"CONTROL\",\"424172388\":\"VARIANT_2\",\"3916828\":\"1\",\"1849602126\":\"1\",\"3465472\":\"1\",\"2142932188\":\"CONTROL\",\"27907428\":\"1\",\"1757398879\":\"CONTROL\",\"309740607\":\"VARIANT\",\"194450372\":\"Control\",\"2138739992\":\"VARIANT\",\"1346406\":\"1\",\"3536353\":\"1\",\"9991951\":\"1\",\"991880274\":\"1\",\"1024569033\":\"1\",\"1705158\":\"1\",\"828711891\":\"VARIANT\",\"1920637096\":\"1\",\"2142738668\":\"1\",\"5161718\":\"1\",\"347637405\":\"CONTROL\",\"2143046971\":\"CONTROL\",\"894468603\":\"1\",\"874894484\":\"1\",\"1052378227\":\"CONTROL\",\"136564865\":\"1\",\"4955452\":\"1\",\"48272024\":\"1\",\"560804523\":\"1\",\"1886980386\":\"1\",\"1209999501\":\"1\",\"1083357638\":\"1\",\"70318012\":\"1\",\"770128680\":\"1\",\"1190054\":\"LINK_AND_FILTER\",\"2144254051\":\"CONTROL\",\"493149231\":\"Variant\",\"881880274\":\"1\",\"1599400529\":\"1\",\"1554845212\":\"1\",\"81888866\":\"1\",\"9952379\":\"1\",\"929067455\":\"Variant\",\"5353708\":\"1\",\"277019363\":\"VARIANT\",\"1468883\":\"1\",\"520689578\":\"Variant\",\"9987874\":\"1\",\"1100876269\":\"1\",\"929943971\":\"1\",\"982890475\":\"1\",\"840906304\":\"1\",\"3485058\":\"1\",\"5328717\":\"CONTROL\",\"1304506957\":\"1\",\"1823011706\":\"NOT_SURE\",\"414609587\":\"1\",\"562400598\":\"1\",\"3290835\":\"1\",\"111628809\":\"1\",\"9321461\":\"1\",\"5691029\":\"1\",\"1939748888\":\"1\",\"103790823\":\"1\",\"901858\":\"1\",\"289794109\":\"1\",\"1295246539\":\"1\",\"7638583\":\"1\",\"1059066909\":\"1\",\"2143430702\":\"CONTROL\",\"9987899\":\"1\",\"1928393\":\"1\",\"1872731713\":\"VARIANT\",\"3026618\":\"1\",\"129135836\":\"VARIANT\",\"1052482575\":\"1\",\"17343434\":\"1\",\"313947703\":\"1\",\"1901186692\":\"CONTROL\",\"118506\":\"1\",\"1901186693\":\"VARIANT\",\"1567924722\":\"1\",\"1901186695\":\"1\",\"7191272\":\"1\",\"1818100572\":\"1\",\"4079785\":\"1\",\"4079786\":\"1\",\"158775426\":\"1\",\"4079784\":\"1\",\"3987210\":\"1\",\"4079782\":\"1\",\"4079780\":\"1\",\"874939014\":\"VARIANT\",\"947361278\":\"1\",\"1148540262\":\"1\",\"1849255435\":\"1\",\"929461\":\"1\",\"32424343\":\"1\",\"9321471\":\"1\",\"1176994901\":\"VARIANT\",\"1520515749\":\"1\",\"78005231\":\"1\",\"6224800\":\"1\",\"270756613\":\"1\",\"1951467539\":\"1\",\"754881256\":\"VARIANT\",\"1013503594\":\"1\",\"2143625861\":\"1\",\"1031170260\":\"1\",\"1336957103\":\"1\",\"1834514030\":\"VARIANT\",\"6190071\":\"1\",\"396076839\":\"CONTROL\",\"4847598\":\"VARIANT\",\"5358563\":\"1\",\"308361658\":\"1\",\"630719918\":\"1\",\"989169531\":\"1\",\"2144049015\":\"VARIANT\",\"1747866071\":\"1\",\"8309156\":\"1\",\"308997399\":\"1\",\"1162275830\":\"Variant\",\"1413505819\":\"1\",\"4079779\":\"1\",\"1438190\":\"1\",\"1689582928\":\"CONTROL\",\"4079777\":\"1\",\"171007336\":\"1\",\"3314447\":\"1\",\"1553789629\":\"VARIANT\",\"2438911\":\"1\",\"1393823267\":\"1\",\"994794180\":\"1\",\"1794278\":\"1\",\"6259530\":\"1\",\"4616886\":\"1\",\"868016154\":\"1\",\"8220014\":\"1\",\"4743765\":\"1\",\"1808403\":\"1\",\"1461291557\":\"1\",\"3191927\":\"1\",\"99274721\":\"1\",\"2144831065\":\"CONTROL\",\"1879652923\":\"VARIANT\",\"1907618601\":\"Variant\",\"5652256\":\"1\",\"546271701\":\"VARIANT\",\"1793544888\":\"Variant\",\"9949501\":\"1\",\"98804786\":\"1\",\"1886770400\":\"1\",\"802032\":\"Update\",\"6231188\":\"1\",\"2119258013\":\"VARIANT\",\"1388579952\":\"1\",\"313304805\":\"1\",\"4270154\":\"1\",\"1550560701\":\"1\",\"2096979930\":\"1\",\"121559624\":\"VARIANT\",\"3886583\":\"1\",\"7837603\":\"1\",\"1671890223\":\"1\",\"182490843\":\"VARIANT\",\"1109737\":\"1\",\"2145927144\":\"CONTROL\",\"1836851391\":\"VARIANT\",\"8103381\":\"1\",\"1025857552\":\"1\",\"1355514593\":\"1\",\"710394\":\"1\",\"1626655\":\"1\",\"1065783\":\"1\",\"3385760\":\"1\",\"1512111093\":\"Variant\",\"4433685\":\"1\",\"504123817\":\"1\",\"1595624026\":\"VARIANT\",\"392725926\":\"Variant\",\"417051630\":\"1\",\"7000114\":\"1\",\"625605953\":\"EMOJIS\",\"1344501680\":\"1\",\"757770628\":\"1\",\"9810319\":\"1\",\"1834671742\":\"1\",\"1901187000\":\"CONTROL\",\"3656274\":\"1\",\"418763486\":\"1\",\"1736618160\":\"1\",\"1600195302\":\"1\",\"1600195301\":\"1\",\"3874561\":\"1\",\"1004676654\":\"1\",\"5802514\":\"1\",\"271225714\":\"VARIANT\",\"1617736\":\"1\",\"4755712\":\"1\",\"5802510\":\"1\",\"12136739\":\"1\",\"1709148411\":\"1\",\"964940211\":\"1\",\"1542421566\":\"1\",\"2160825\":\"1\",\"1526306247\":\"CONTROL\",\"1321161145\":\"1\",\"854594032\":\"Variant\",\"2041825578\":\"1\",\"11904\":\"1\",\"6217731\":\"1\",\"247697721\":\"1\",\"1619492\":\"1\",\"2105513715\":\"1\",\"2273621\":\"1\",\"168753\":\"1\",\"923170641\":\"CONTROL\",\"382165445\":\"1\",\"3224441\":\"1\",\"7982568\":\"1\",\"3144772\":\"1\",\"1567065710\":\"1\",\"237030250\":\"Control\",\"9201525\":\"1\",\"1389482332\":\"VARIANT\",\"7810534\":\"1\",\"4632039\":\"1\",\"1837828\":\"1\",\"757751552\":\"CONTROL\",\"4632036\":\"New_Saved_Searches_Page\",\"6480638\":\"1\",\"6481960\":\"1\",\"813904459\":\"1\",\"1262243488\":\"1\",\"4097541\":\"1\",\"97794078\":\"1\",\"279081714\":\"Variant\",\"315377577\":\"1\",\"2145620513\":\"1\",\"1158219574\":\"1\",\"129019436\":\"1\",\"462723816\":\"1\",\"18231134\":\"1\",\"106440076\":\"1\",\"217351236\":\"POST_LOGIN_PROMPT\",\"1463942\":\"1\",\"6163260\":\"CONTROL\",\"624744214\":\"Variant\",\"6163261\":\"1\",\"1684611024\":\"1\",\"8295610\":\"1\",\"9389938\":\"1\",\"1432236952\":\"1\",\"3121244\":\"1\",\"129543451\":\"1\",\"680484385\":\"1\",\"1213362557\":\"1\",\"986912\":\"VARIANT\",\"2143815247\":\"CONTROL\",\"1451978\":\"1\",\"1105907132\":\"1\",\"1358911041\":\"Control\",\"7600870\":\"1\",\"1532748560\":\"VARIANT_2\",\"1455565429\":\"1\",\"784360524\":\"Variant_A\",\"1513346880\":\"CONTROL\",\"1059070\":\"1\",\"8992517\":\"1\",\"4132566\":\"1\",\"243322139\":\"1\",\"7885488\":\"1\",\"2144297268\":\"1\",\"333562983\":\"1\",\"9866710\":\"1\",\"303007370\":\"1\",\"795020313\":\"1\",\"6885496\":\"1\",\"1517451707\":\"1\",\"2041672266\":\"1\",\"1913163\":\"1\",\"403201958\":\"1\",\"8873936\":\"1\",\"3823664\":\"1\",\"104226690\":\"Variant\",\"6002850\":\"1\",\"464166739\":\"CONTROL\",\"1527868500\":\"1\",\"1154635776\":\"HAPTIC_PLUS_ANIMATIONS\",\"38038763\":\"1\",\"1815478023\":\"Variant\",\"2932326\":\"1\",\"1337636259\":\"1\",\"791875\":\"1\",\"2686905\":\"1\",\"904897\":\"1\",\"756397708\":\"VARIANT\",\"500441437\":\"Variant B\",\"452225222\":\"1\",\"736917836\":\"1\",\"2143712872\":\"CONTROL\",\"682661987\":\"1\",\"1769205491\":\"1\",\"1691714049\":\"VARIANT\",\"1834175836\":\"1\",\"1456677671\":\"1\",\"1004628075\":\"CONTROL\",\"4413346\":\"1\",\"903739094\":\"1\",\"2032398958\":\"Variant\",\"91343540\":\"1\",\"1435743057\":\"1\",\"8424835\":\"Control\",\"505306807\":\"1\",\"1318068823\":\"1\",\"34786210\":\"VARIANT\",\"565632833\":\"1\",\"4079057\":\"1\",\"4079058\":\"1\",\"942398417\":\"VARIANT\",\"413586466\":\"VARIANT\",\"829956152\":\"VARIANT_1\",\"6300307\":\"1\",\"1415329727\":\"1\",\"31645138\":\"1\",\"2384457\":\"VARIANT\",\"915571\":\"VARIANT\",\"6674243\":\"Control\",\"544678836\":\"VARIANT\",\"1898013599\":\"1\",\"2145499369\":\"1\",\"130921229\":\"1\",\"210916\":\"1\",\"1716610413\":\"1\",\"48052913\":\"1\",\"4247731\":\"1\",\"525625380\":\"1\",\"1390014443\":\"1\",\"633893488\":\"Variant\",\"837042329\":\"1\",\"1266675492\":\"1\",\"939101826\":\"Variant\",\"4996092\":\"1\",\"7334181\":\"1\",\"215921446\":\"CONTROL\",\"9830832\":\"1\",\"1998007633\":\"1\",\"676347271\":\"VARIANT\",\"575914\":\"1\",\"353503658\":\"1\",\"1497793053\":\"1\",\"711121082\":\"VARIANT\",\"975638567\":\"1\",\"3973375\":\"1\",\"593469827\":\"VARIANT\",\"1903044102\":\"1\",\"9481474\":\"1\",\"2144852179\":\"VARIANT_TIMEPICKER\",\"1134554027\":\"1\",\"1790756\":\"1\",\"4602072\":\"1\",\"1463503520\":\"1\",\"1980052012\":\"1\",\"1452454356\":\"1\",\"1789325874\":\"1\",\"1836041904\":\"CONTROL\",\"1396593632\":\"1\",\"745003478\":\"1\",\"211736921\":\"1\",\"94363479\":\"1\",\"1962252465\":\"1\",\"716672882\":\"1\",\"94363475\":\"1\",\"94363474\":\"1\",\"94363473\":\"1\",\"1307052112\":\"1\",\"1693181669\":\"1\",\"7181768\":\"1\",\"2066069\":\"VARIANT\",\"1714074476\":\"VARIANT\",\"960715691\":\"1\",\"1965311699\":\"VARIANT\",\"3588253\":\"1\",\"1762815729\":\"CONTROL\",\"1716834\":\"1\",\"1716833\":\"1\",\"3746316\":\"1\",\"767146429\":\"CONTROL\",\"6233883\":\"1\",\"2779102\":\"1\",\"7192428\":\"1\",\"6743522\":\"1\",\"742055\":\"1\",\"279921070\":\"1\",\"1246605019\":\"1\",\"1021728712\":\"1\",\"5134262\":\"WITH_PAGINATION\",\"2513992\":\"1\",\"6889187\":\"1\",\"1263414384\":\"1\",\"7009541\":\"1\",\"277201\":\"VARIANT\",\"8438674\":\"1\",\"1639533699\":\"1\",\"1141629680\":\"1\",\"5788430\":\"VARIANT\",\"1096706453\":\"1\",\"1639820292\":\"1\",\"3845088\":\"1\",\"1330271695\":\"Variant\",\"2083997629\":\"1\",\"1928667375\":\"VARIANT\",\"1650604996\":\"1\",\"4488953\":\"1\",\"1127570\":\"1\",\…')
b = '{"ReactServerAgent.cache":{"dataCache":{"\u002Fstingray\u002Fcorvair\u002Fv2\u002FpreresolveBouncerFlags":{"url":"\u002Fstingray\u002Fcorvair\u002Fv2\u002FpreresolveBouncerFlags","requestData":{"urlPath":"\u002Fstingray\u002Fcorvair\u002Fv2\u002FpreresolveBouncerFlags","method":"GET","queryParams":[],"postParams":null,"type":"json"},"requesters":2,"loaded":true,"res":{"_hasBody":true,"text":"{}&&{\"bouncerData\":{\"4496425\":\"1\",\"863218446\":\"1\",\"8537661\":\"1\",\"267639697\":\"Variant\",\"2428670\":\"1\",\"1203506167\":\"1\",\"967225084\":\"1\",\"52278200\":\"1\",\"1016995215\":\"1\",\"876931639\":\"1\",\"1455831115\":\"1\",\"1545207106\":\"1\",\"4\":\"1\",\"79525\":\"1\",\"9565569\":\"1\",\"1714458391\":\"VARIANT\",\"2068074617\":\"1\",\"988347912\":\"1\",\"381469020\":\"1\",\"621081811\":\"VARIANT\",\"324331314\":\"1\",\"128514355\":\"1\",\"1610565413\":\"1\",\"1584385382\":\"Variant\",\"1908214897\":\"1\",\"1210857133\":\"Variant\",\"6347749\":\"1\",\"902143405\":\"1\",\"4455910\":\"CONTROL\",\"424172388\":\"VARIANT_2\",\"3916828\":\"1\",\"1849602126\":\"1\",\"3465472\":\"1\",\"2142932188\":\"CONTROL\",\"1757398879\":\"CONTROL\",\"27907428\":\"1\",\"309740607\":\"VARIANT\",\"194450372\":\"Control\",\"2138739992\":\"VARIANT\",\"1346406\":\"1\",\"3536353\":\"1\",\"9991951\":\"1\",\"991880274\":\"1\",\"1024569033\":\"1\",\"1705158\":\"1\",\"828711891\":\"VARIANT\",\"1920637096\":\"1\",\"2142738668\":\"1\",\"5161718\":\"1\",\"347637405\":\"CONTROL\",\"2143046971\":\"CONTROL\",\"894468603\":\"1\",\"874894484\":\"1\",\"1052378227\":\"CONTROL\",\"136564865\":\"1\",\"4955452\":\"1\",\"48272024\":\"1\",\"560804523\":\"1\",\"1886980386\":\"1\",\"1209999501\":\"1\",\"1083357638\":\"1\",\"70318012\":\"1\",\"1190054\":\"LINK_AND_FILTER\",\"770128680\":\"1\",\"2144254051\":\"CONTROL\",\"493149231\":\"Variant\",\"881880274\":\"1\",\"1599400529\":\"1\",\"1554845212\":\"1\",\"81888866\":\"1\",\"9952379\":\"1\",\"929067455\":\"Variant\",\"5353708\":\"1\",\"277019363\":\"VARIANT\",\"1468883\":\"1\",\"520689578\":\"Variant\",\"9987874\":\"1\",\"1100876269\":\"1\",\"929943971\":\"1\",\"982890475\":\"1\",\"840906304\":\"1\",\"3485058\":\"1\",\"5328717\":\"CONTROL\",\"1304506957\":\"1\",\"1823011706\":\"NOT_SURE\",\"414609587\":\"1\",\"562400598\":\"1\",\"3290835\":\"1\",\"111628809\":\"1\",\"5691029\":\"1\",\"9321461\":\"1\",\"1939748888\":\"1\",\"103790823\":\"1\",\"901858\":\"1\",\"289794109\":\"1\",\"1295246539\":\"1\",\"7638583\":\"1\",\"1059066909\":\"1\",\"2143430702\":\"CONTROL\",\"1928393\":\"1\",\"9987899\":\"1\",\"1872731713\":\"VARIANT\",\"3026618\":\"1\",\"129135836\":\"VARIANT\",\"1052482575\":\"1\",\"17343434\":\"1\",\"313947703\":\"1\",\"1901186692\":\"CONTROL\",\"118506\":\"1\",\"1901186693\":\"VARIANT\",\"1567924722\":\"1\",\"1901186695\":\"1\",\"7191272\":\"1\",\"1818100572\":\"1\",\"4079785\":\"1\",\"158775426\":\"1\",\"4079786\":\"1\",\"4079784\":\"1\",\"3987210\":\"1\",\"4079782\":\"1\",\"4079780\":\"1\",\"874939014\":\"VARIANT\",\"947361278\":\"1\",\"1148540262\":\"1\",\"1849255435\":\"1\",\"929461\":\"1\",\"32424343\":\"1\",\"9321471\":\"1\",\"1176994901\":\"VARIANT\",\"1520515749\":\"1\",\"78005231\":\"1\",\"6224800\":\"1\",\"270756613\":\"1\",\"1951467539\":\"1\",\"754881256\":\"VARIANT\",\"1013503594\":\"1\",\"2143625861\":\"1\",\"1031170260\":\"1\",\"1336957103\":\"1\",\"6190071\":\"1\",\"1834514030\":\"VARIANT\",\"396076839\":\"CONTROL\",\"4847598\":\"VARIANT\",\"5358563\":\"1\",\"308361658\":\"1\",\"630719918\":\"1\",\"989169531\":\"1\",\"2144049015\":\"VARIANT\",\"1747866071\":\"1\",\"8309156\":\"1\",\"308997399\":\"1\",\"1162275830\":\"Variant\",\"1413505819\":\"1\",\"4079779\":\"1\",\"1689582928\":\"CONTROL\",\"1438190\":\"1\",\"4079777\":\"1\",\"171007336\":\"1\",\"3314447\":\"1\",\"1553789629\":\"VARIANT\",\"2438911\":\"1\",\"1393823267\":\"1\",\"994794180\":\"1\",\"1794278\":\"1\",\"4616886\":\"1\",\"6259530\":\"1\",\"868016154\":\"1\",\"8220014\":\"1\",\"4743765\":\"1\",\"1808403\":\"1\",\"1461291557\":\"1\",\"3191927\":\"1\",\"99274721\":\"1\",\"2144831065\":\"CONTROL\",\"1879652923\":\"VARIANT\",\"1907618601\":\"Variant\",\"5652256\":\"1\",\"1793544888\":\"Variant\",\"546271701\":\"VARIANT\",\"9949501\":\"1\",\"98804786\":\"1\",\"1886770400\":\"1\",\"802032\":\"Update\",\"6231188\":\"1\",\"2119258013\":\"VARIANT\",\"1388579952\":\"1\",\"313304805\":\"1\",\"4270154\":\"1\",\"1550560701\":\"1\",\"2096979930\":\"1\",\"121559624\":\"VARIANT\",\"3886583\":\"1\",\"7837603\":\"1\",\"1671890223\":\"1\",\"1109737\":\"1\",\"182490843\":\"VARIANT\",\"2145927144\":\"CONTROL\",\"1836851391\":\"VARIANT\",\"8103381\":\"1\",\"1025857552\":\"1\",\"1355514593\":\"1\",\"710394\":\"1\",\"1626655\":\"1\",\"1065783\":\"1\",\"3385760\":\"1\",\"1512111093\":\"Variant\",\"4433685\":\"1\",\"504123817\":\"1\",\"1595624026\":\"VARIANT\",\"392725926\":\"Variant\",\"417051630\":\"1\",\"7000114\":\"1\",\"625605953\":\"EMOJIS\",\"1344501680\":\"1\",\"757770628\":\"1\",\"9810319\":\"1\",\"1834671742\":\"1\",\"1901187000\":\"CONTROL\",\"3656274\":\"1\",\"418763486\":\"1\",\"1600195302\":\"1\",\"1736618160\":\"1\",\"1600195301\":\"1\",\"3874561\":\"1\",\"1004676654\":\"1\",\"5802514\":\"1\",\"271225714\":\"VARIANT\",\"1617736\":\"1\",\"4755712\":\"1\",\"5802510\":\"1\",\"12136739\":\"1\",\"1709148411\":\"1\",\"964940211\":\"1\",\"1542421566\":\"1\",\"2160825\":\"1\",\"1526306247\":\"CONTROL\",\"1321161145\":\"1\",\"854594032\":\"Variant\",\"2041825578\":\"1\",\"11904\":\"1\",\"6217731\":\"1\",\"247697721\":\"1\",\"1619492\":\"1\",\"2105513715\":\"1\",\"2273621\":\"1\",\"168753\":\"1\",\"923170641\":\"CONTROL\",\"382165445\":\"1\",\"3224441\":\"1\",\"7982568\":\"1\",\"3144772\":\"1\",\"1567065710\":\"1\",\"237030250\":\"Control\",\"9201525\":\"1\",\"1389482332\":\"VARIANT\",\"7810534\":\"1\",\"4632039\":\"1\",\"1837828\":\"1\",\"757751552\":\"CONTROL\",\"6480638\":\"1\",\"4632036\":\"New_Saved_Searches_Page\",\"6481960\":\"1\",\"813904459\":\"1\",\"1262243488\":\"1\",\"4097541\":\"1\",\"97794078\":\"1\",\"279081714\":\"Variant\",\"315377577\":\"1\",\"2145620513\":\"1\",\"1158219574\":\"1\",\"129019436\":\"1\",\"462723816\":\"1\",\"106440076\":\"1\",\"18231134\":\"1\",\"217351236\":\"POST_LOGIN_PROMPT\",\"1463942\":\"1\",\"6163260\":\"CONTROL\",\"6163261\":\"1\",\"624744214\":\"Variant\",\"1684611024\":\"1\",\"8295610\":\"1\",\"9389938\":\"1\",\"1432236952\":\"1\",\"3121244\":\"1\",\"129543451\":\"1\",\"680484385\":\"1\",\"1213362557\":\"1\",\"986912\":\"VARIANT\",\"2143815247\":\"CONTROL\",\"1451978\":\"1\",\"1105907132\":\"1\",\"1358911041\":\"Control\",\"7600870\":\"1\",\"1532748560\":\"VARIANT_2\",\"1455565429\":\"1\",\"784360524\":\"Variant_A\",\"1513346880\":\"CONTROL\",\"1059070\":\"1\",\"8992517\":\"1\",\"4132566\":\"1\",\"7885488\":\"1\",\"243322139\":\"1\",\"2144297268\":\"1\",\"333562983\":\"1\",\"9866710\":\"1\",\"303007370\":\"1\",\"795020313\":\"1\",\"1517451707\":\"1\",\"6885496\":\"1\",\"2041672266\":\"1\",\"1913163\":\"1\",\"403201958\":\"1\",\"3823664\":\"1\",\"8873936\":\"1\",\"104226690\":\"Variant\",\"6002850\":\"1\",\"464166739\":\"CONTROL\",\"1527868500\":\"1\",\"1154635776\":\"HAPTIC_PLUS_ANIMATIONS\",\"38038763\":\"1\",\"1815478023\":\"Variant\",\"2932326\":\"1\",\"1337636259\":\"1\",\"791875\":\"1\",\"2686905\":\"1\",\"904897\":\"1\",\"756397708\":\"VARIANT\",\"500441437\":\"Variant B\",\"736917836\":\"1\",\"452225222\":\"1\",\"682661987\":\"1\",\"2143712872\":\"CONTROL\",\"1769205491\":\"1\",\"1691714049\":\"VARIANT\",\"1834175836\":\"1\",\"1456677671\":\"1\",\"1004628075\":\"CONTROL\",\"4413346\":\"1\",\"903739094\":\"1\",\"2032398958\":\"Variant\",\"91343540\":\"1\",\"1435743057\":\"1\",\"8424835\":\"Control\",\"505306807\":\"1\",\"1318068823\":\"1\",\"34786210\":\"VARIANT\",\"565632833\":\"1\",\"4079057\":\"1\",\"4079058\":\"1\",\"942398417\":\"VARIANT\",\"413586466\":\"VARIANT\",\"829956152\":\"VARIANT_1\",\"6300307\":\"1\",\"1415329727\":\"1\",\"2384457\":\"VARIANT\",\"31645138\":\"1\",\"915571\":\"VARIANT\",\"6674243\":\"Control\",\"544678836\":\"VARIANT\",\"1898013599\":\"1\",\"2145499369\":\"1\",\"130921229\":\"1\",\"210916\":\"1\",\"1716610413\":\"1\",\"48052913\":\"1\",\"4247731\":\"1\",\"525625380\":\"1\",\"633893488\":\"Variant\",\"1390014443\":\"1\",\"837042329\":\"1\",\"1266675492\":\"1\",\"939101826\":\"Variant\",\"4996092\":\"1\",\"7334181\":\"1\",\"215921446\":\"CONTROL\",\"9830832\":\"1\",\"1998007633\":\"1\",\"676347271\":\"VARIANT\",\"575914\":\"1\",\"353503658\":\"1\",\"1497793053\":\"1\",\"711121082\":\"VARIANT\",\"975638567\":\"1\",\"3973375\":\"1\",\"593469827\":\"VARIANT\",\"1903044102\":\"1\",\"9481474\":\"1\",\"2144852179\":\"VARIANT_TIMEPICKER\",\"1134554027\":\"1\",\"1790756\":\"1\",\"4602072\":\"1\",\"1463503520\":\"1\",\"1980052012\":\"1\",\"1452454356\":\"1\",\"1789325874\":\"1\",\"1396593632\":\"1\",\"1836041904\":\"CONTROL\",\"745003478\":\"1\",\"211736921\":\"1\",\"94363479\":\"1\",\"716672882\":\"1\",\"1962252465\":\"1\",\"94363475\":\"1\",\"94363474\":\"1\",\"94363473\":\"1\",\"1307052112\":\"1\",\"1693181669\":\"1\",\"7181768\":\"1\",\"1714074476\":\"VARIANT\",\"2066069\":\"VARIANT\",\"960715691\":\"1\",\"1965311699\":\"VARIANT\",\"1762815729\":\"CONTROL\",\"3588253\":\"1\",\"1716834\":\"1\",\"1716833\":\"1\",\"3746316\":\"1\",\"767146429\":\"CONTROL\",\"6233883\":\"1\",\"2779102\":\"1\",\"7192428\":\"1\",\"6743522\":\"1\",\"742055\":\"1\",\"279921070\":\"1\",\"1246605019\":\"1\",\"1021728712\":\"1\",\"5134262\":\"WITH_PAGINATION\",\"2513992\":\"1\",\"6889187\":\"1\",\"1263414384\":\"1\",\"7009541\":\"1\",\"277201\":\"VARIANT\",\"8438674\":\"1\",\"1639533699\":\"1\",\"1141629680\":\"1\",\"5788430\":\"VARIANT\",\"1096706453\":\"1\",\"1639820292\":\"1\",\"3845088\":\"1\",\"1330271695\":\"Variant\",\"2083997629\":\"1\",\"1928667375\":\"VARIANT\",\"1650604996\":\"1\",\"4488953\":\"1\",\"1127570\":\"1\",\…'
c = '{"ReactServerAgent.cache":{"dataCache":{"\u002Fstingray\u002Fcorvair\u002Fv2\u002FpreresolveBouncerFlags":{"url":"\u002Fstingray\u002Fcorvair\u002Fv2\u002FpreresolveBouncerFlags","requestData":{"urlPath":"\u002Fstingray\u002Fcorvair\u002Fv2\u002FpreresolveBouncerFlags","method":"GET","queryParams":[],"postParams":null,"type":"json"},"requesters":2,"loaded":true,"res":{"_hasBody":true,"text":"{}&&{\"bouncerData\":{\"4496425\":\"1\",\"863218446\":\"1\",\"8537661\":\"1\",\"267639697\":\"Variant\",\"2428670\":\"1\",\"1203506167\":\"1\",\"967225084\":\"1\",\"52278200\":\"1\",\"1016995215\":\"1\",\"876931639\":\"1\",\"1455831115\":\"1\",\"1545207106\":\"1\",\"4\":\"1\",\"79525\":\"1\",\"9565569\":\"1\",\"1714458391\":\"VARIANT\",\"2068074617\":\"1\",\"988347912\":\"1\",\"381469020\":\"1\",\"621081811\":\"VARIANT\",\"324331314\":\"1\",\"128514355\":\"1\",\"1610565413\":\"1\",\"1584385382\":\"Variant\",\"1908214897\":\"1\",\"1210857133\":\"Variant\",\"6347749\":\"1\",\"902143405\":\"1\",\"4455910\":\"CONTROL\",\"424172388\":\"VARIANT_2\",\"3916828\":\"1\",\"1849602126\":\"1\",\"3465472\":\"1\",\"2142932188\":\"CONTROL\",\"27907428\":\"1\",\"1757398879\":\"CONTROL\",\"309740607\":\"VARIANT\",\"194450372\":\"Control\",\"2138739992\":\"VARIANT\",\"1346406\":\"1\",\"3536353\":\"1\",\"991880274\":\"1\",\"9991951\":\"1\",\"1024569033\":\"1\",\"1705158\":\"1\",\"828711891\":\"VARIANT\",\"1920637096\":\"1\",\"5161718\":\"1\",\"2142738668\":\"1\",\"347637405\":\"CONTROL\",\"2143046971\":\"CONTROL\",\"894468603\":\"1\",\"874894484\":\"1\",\"1052378227\":\"CONTROL\",\"136564865\":\"1\",\"4955452\":\"1\",\"48272024\":\"1\",\"560804523\":\"1\",\"1886980386\":\"1\",\"1209999501\":\"1\",\"1083357638\":\"1\",\"70318012\":\"1\",\"1190054\":\"LINK_AND_FILTER\",\"770128680\":\"1\",\"2144254051\":\"CONTROL\",\"493149231\":\"Variant\",\"881880274\":\"1\",\"1599400529\":\"1\",\"1554845212\":\"1\",\"81888866\":\"1\",\"9952379\":\"1\",\"929067455\":\"Variant\",\"5353708\":\"1\",\"277019363\":\"VARIANT\",\"1468883\":\"1\",\"520689578\":\"Variant\",\"9987874\":\"1\",\"1100876269\":\"1\",\"929943971\":\"1\",\"982890475\":\"1\",\"840906304\":\"1\",\"3485058\":\"1\",\"5328717\":\"CONTROL\",\"1304506957\":\"1\",\"1823011706\":\"NOT_SURE\",\"414609587\":\"1\",\"562400598\":\"1\",\"3290835\":\"1\",\"111628809\":\"1\",\"5691029\":\"1\",\"9321461\":\"1\",\"1939748888\":\"1\",\"103790823\":\"1\",\"901858\":\"1\",\"289794109\":\"1\",\"1295246539\":\"1\",\"7638583\":\"1\",\"1059066909\":\"1\",\"2143430702\":\"CONTROL\",\"9987899\":\"1\",\"1928393\":\"1\",\"1872731713\":\"VARIANT\",\"129135836\":\"VARIANT\",\"3026618\":\"1\",\"1052482575\":\"1\",\"17343434\":\"1\",\"313947703\":\"1\",\"1901186692\":\"CONTROL\",\"118506\":\"1\",\"1901186693\":\"VARIANT\",\"1567924722\":\"1\",\"1901186695\":\"1\",\"7191272\":\"1\",\"4079785\":\"1\",\"1818100572\":\"1\",\"158775426\":\"1\",\"4079786\":\"1\",\"4079784\":\"1\",\"3987210\":\"1\",\"4079782\":\"1\",\"4079780\":\"1\",\"874939014\":\"VARIANT\",\"947361278\":\"1\",\"1148540262\":\"1\",\"1849255435\":\"1\",\"929461\":\"1\",\"32424343\":\"1\",\"9321471\":\"1\",\"1176994901\":\"VARIANT\",\"1520515749\":\"1\",\"78005231\":\"1\",\"6224800\":\"1\",\"1951467539\":\"1\",\"270756613\":\"1\",\"754881256\":\"VARIANT\",\"1013503594\":\"1\",\"2143625861\":\"1\",\"1031170260\":\"1\",\"1336957103\":\"1\",\"1834514030\":\"VARIANT\",\"6190071\":\"1\",\"396076839\":\"CONTROL\",\"5358563\":\"1\",\"4847598\":\"VARIANT\",\"308361658\":\"1\",\"630719918\":\"1\",\"989169531\":\"1\",\"2144049015\":\"VARIANT\",\"1747866071\":\"1\",\"8309156\":\"1\",\"308997399\":\"1\",\"1162275830\":\"Variant\",\"1413505819\":\"1\",\"4079779\":\"1\",\"1689582928\":\"CONTROL\",\"1438190\":\"1\",\"4079777\":\"1\",\"171007336\":\"1\",\"3314447\":\"1\",\"1553789629\":\"VARIANT\",\"2438911\":\"1\",\"1393823267\":\"1\",\"994794180\":\"1\",\"1794278\":\"1\",\"6259530\":\"1\",\"4616886\":\"1\",\"868016154\":\"1\",\"8220014\":\"1\",\"4743765\":\"1\",\"1808403\":\"1\",\"1461291557\":\"1\",\"3191927\":\"1\",\"99274721\":\"1\",\"2144831065\":\"CONTROL\",\"1879652923\":\"VARIANT\",\"1907618601\":\"Variant\",\"5652256\":\"1\",\"546271701\":\"VARIANT\",\"1793544888\":\"Variant\",\"9949501\":\"1\",\"98804786\":\"1\",\"1886770400\":\"1\",\"802032\":\"Update\",\"6231188\":\"1\",\"2119258013\":\"VARIANT\",\"1388579952\":\"1\",\"313304805\":\"1\",\"4270154\":\"1\",\"1550560701\":\"1\",\"2096979930\":\"1\",\"121559624\":\"VARIANT\",\"3886583\":\"1\",\"7837603\":\"1\",\"1671890223\":\"1\",\"182490843\":\"VARIANT\",\"1109737\":\"1\",\"2145927144\":\"CONTROL\",\"1836851391\":\"VARIANT\",\"8103381\":\"1\",\"1025857552\":\"1\",\"1355514593\":\"1\",\"710394\":\"1\",\"1626655\":\"1\",\"1065783\":\"1\",\"3385760\":\"1\",\"1512111093\":\"Variant\",\"4433685\":\"1\",\"504123817\":\"1\",\"1595624026\":\"VARIANT\",\"392725926\":\"Variant\",\"417051630\":\"1\",\"7000114\":\"1\",\"625605953\":\"EMOJIS\",\"1344501680\":\"1\",\"757770628\":\"1\",\"9810319\":\"1\",\"1834671742\":\"1\",\"1901187000\":\"CONTROL\",\"3656274\":\"1\",\"418763486\":\"1\",\"1600195302\":\"1\",\"1736618160\":\"1\",\"1600195301\":\"1\",\"3874561\":\"1\",\"1004676654\":\"1\",\"5802514\":\"1\",\"271225714\":\"VARIANT\",\"1617736\":\"1\",\"4755712\":\"1\",\"5802510\":\"1\",\"12136739\":\"1\",\"1709148411\":\"1\",\"964940211\":\"1\",\"1542421566\":\"1\",\"2160825\":\"1\",\"1321161145\":\"1\",\"1526306247\":\"CONTROL\",\"854594032\":\"Variant\",\"2041825578\":\"1\",\"11904\":\"1\",\"6217731\":\"1\",\"247697721\":\"1\",\"1619492\":\"1\",\"2105513715\":\"1\",\"2273621\":\"1\",\"168753\":\"1\",\"923170641\":\"CONTROL\",\"382165445\":\"1\",\"3224441\":\"1\",\"7982568\":\"1\",\"3144772\":\"1\",\"1567065710\":\"1\",\"237030250\":\"Control\",\"9201525\":\"1\",\"7810534\":\"1\",\"1389482332\":\"VARIANT\",\"4632039\":\"1\",\"1837828\":\"1\",\"6480638\":\"1\",\"4632036\":\"New_Saved_Searches_Page\",\"757751552\":\"CONTROL\",\"6481960\":\"1\",\"813904459\":\"1\",\"1262243488\":\"1\",\"4097541\":\"1\",\"97794078\":\"1\",\"279081714\":\"Variant\",\"315377577\":\"1\",\"2145620513\":\"1\",\"1158219574\":\"1\",\"129019436\":\"1\",\"462723816\":\"1\",\"106440076\":\"1\",\"18231134\":\"1\",\"1463942\":\"1\",\"217351236\":\"POST_LOGIN_PROMPT\",\"6163260\":\"CONTROL\",\"6163261\":\"1\",\"624744214\":\"Variant\",\"1684611024\":\"1\",\"8295610\":\"1\",\"9389938\":\"1\",\"1432236952\":\"1\",\"3121244\":\"1\",\"129543451\":\"1\",\"680484385\":\"1\",\"1213362557\":\"1\",\"2143815247\":\"CONTROL\",\"986912\":\"VARIANT\",\"1451978\":\"1\",\"1105907132\":\"1\",\"1358911041\":\"Control\",\"7600870\":\"1\",\"1532748560\":\"VARIANT_2\",\"1455565429\":\"1\",\"784360524\":\"Variant_A\",\"1513346880\":\"CONTROL\",\"1059070\":\"1\",\"8992517\":\"1\",\"4132566\":\"1\",\"243322139\":\"1\",\"2144297268\":\"1\",\"7885488\":\"1\",\"333562983\":\"1\",\"9866710\":\"1\",\"303007370\":\"1\",\"795020313\":\"1\",\"1517451707\":\"1\",\"6885496\":\"1\",\"2041672266\":\"1\",\"1913163\":\"1\",\"403201958\":\"1\",\"3823664\":\"1\",\"8873936\":\"1\",\"104226690\":\"Variant\",\"6002850\":\"1\",\"464166739\":\"CONTROL\",\"1527868500\":\"1\",\"1154635776\":\"HAPTIC_PLUS_ANIMATIONS\",\"38038763\":\"1\",\"1815478023\":\"Variant\",\"2932326\":\"1\",\"1337636259\":\"1\",\"791875\":\"1\",\"2686905\":\"1\",\"904897\":\"1\",\"756397708\":\"VARIANT\",\"500441437\":\"Variant B\",\"736917836\":\"1\",\"452225222\":\"1\",\"682661987\":\"1\",\"2143712872\":\"CONTROL\",\"1769205491\":\"1\",\"1691714049\":\"VARIANT\",\"1834175836\":\"1\",\"1456677671\":\"1\",\"1004628075\":\"CONTROL\",\"4413346\":\"1\",\"903739094\":\"1\",\"2032398958\":\"Variant\",\"91343540\":\"1\",\"1435743057\":\"1\",\"8424835\":\"Control\",\"505306807\":\"1\",\"1318068823\":\"1\",\"34786210\":\"VARIANT\",\"565632833\":\"1\",\"4079057\":\"1\",\"4079058\":\"1\",\"942398417\":\"VARIANT\",\"413586466\":\"VARIANT\",\"829956152\":\"VARIANT_1\",\"6300307\":\"1\",\"1415329727\":\"1\",\"2384457\":\"VARIANT\",\"31645138\":\"1\",\"915571\":\"VARIANT\",\"6674243\":\"Control\",\"544678836\":\"VARIANT\",\"2145499369\":\"1\",\"1898013599\":\"1\",\"130921229\":\"1\",\"210916\":\"1\",\"48052913\":\"1\",\"1716610413\":\"1\",\"4247731\":\"1\",\"525625380\":\"1\",\"633893488\":\"Variant\",\"1390014443\":\"1\",\"837042329\":\"1\",\"1266675492\":\"1\",\"939101826\":\"Variant\",\"4996092\":\"1\",\"7334181\":\"1\",\"215921446\":\"CONTROL\",\"9830832\":\"1\",\"1998007633\":\"1\",\"676347271\":\"VARIANT\",\"353503658\":\"1\",\"575914\":\"1\",\"1497793053\":\"1\",\"711121082\":\"VARIANT\",\"975638567\":\"1\",\"3973375\":\"1\",\"593469827\":\"VARIANT\",\"1903044102\":\"1\",\"9481474\":\"1\",\"1134554027\":\"1\",\"2144852179\":\"VARIANT_TIMEPICKER\",\"1790756\":\"1\",\"4602072\":\"1\",\"1463503520\":\"1\",\"1980052012\":\"1\",\"1452454356\":\"1\",\"1789325874\":\"1\",\"1396593632\":\"1\",\"1836041904\":\"CONTROL\",\"745003478\":\"1\",\"211736921\":\"1\",\"94363479\":\"1\",\"716672882\":\"1\",\"1962252465\":\"1\",\"94363475\":\"1\",\"94363474\":\"1\",\"1307052112\":\"1\",\"94363473\":\"1\",\"7181768\":\"1\",\"1693181669\":\"1\",\"2066069\":\"VARIANT\",\"1714074476\":\"VARIANT\",\"960715691\":\"1\",\"1965311699\":\"VARIANT\",\"3588253\":\"1\",\"1762815729\":\"CONTROL\",\"1716834\":\"1\",\"1716833\":\"1\",\"3746316\":\"1\",\"767146429\":\"CONTROL\",\"6233883\":\"1\",\"2779102\":\"1\",\"7192428\":\"1\",\"6743522\":\"1\",\"742055\":\"1\",\"279921070\":\"1\",\"1246605019\":\"1\",\"1021728712\":\"1\",\"5134262\":\"WITH_PAGINATION\",\"2513992\":\"1\",\"6889187\":\"1\",\"1263414384\":\"1\",\"7009541\":\"1\",\"277201\":\"VARIANT\",\"8438674\":\"1\",\"1639533699\":\"1\",\"1141629680\":\"1\",\"5788430\":\"VARIANT\",\"1096706453\":\"1\",\"1639820292\":\"1\",\"3845088\":\"1\",\"1330271695\":\"Variant\",\"2083997629\":\"1\",\"1928667375\":\"VARIANT\",\"1650604996\":\"1\",\"4488953\":\"1\",\"1127570\":\"1\",\…'
import difflib

for d in difflib.ndiff(a,b):
    if d[0] == '-' or d[0] == '+':
        pass  # print(d)
re.match(f'%s{e<=40}'%a, b)

JSONDecodeError: Expecting ',' delimiter: line 1 column 342 (char 341)

In [278]:
b[200:450]

'cerFlags","method":"GET","queryParams":[],"postParams":null,"type":"json"},"requesters":2,"loaded":true,"res":{"_hasBody":true,"text":"{}&&{"bouncerData":{"4496425":"1","863218446":"1","8537661":"1","267639697":"Variant","2428670":"1","1203506167":"1'

In [183]:
df[df.city == 'Upper St. Clair'].head()

,showMlsId,mlsStatus,showDatasourceLogo,hideSalePrice,isHoaFrequencyKnown,beds,baths,fullBaths,city,state,...,listingAgent.name,listingAgent.redfinAgentId,partialBaths,stories,openHouseStart,openHouseEnd,openHouseStartFormatted,openHouseEventName,listingBroker.phone,buildingId
propertyId,,,,,,,,,,,,,,,,,,,,,
74587279,False,Active,True,False,True,5,4.500,4.000,Upper St. Clair,PA,...,NaN,NaN,1.000,2.000,"1,731,783,600,000.000","1,731,790,800,000.000","Nov 16, 2:00PM",Open House - 2:00 - 4:00 PM,NaN,NaN
74681956,False,Active,True,False,True,3,2.500,2.000,Upper St. Clair,PA,...,NaN,NaN,1.000,NaN,"1,731,772,800,000.000","1,731,780,000,000.000","Nov 16, 11:00AM",Open House - 11:00 - 1:00 PM,NaN,NaN
74681704,False,Active,True,False,True,3,2.500,2.000,Upper St. Clair,PA,...,NaN,NaN,1.000,2.000,NaN,NaN,NaN,NaN,NaN,NaN
74587279,False,Active,True,False,True,5,4.500,4.000,Upper St. Clair,PA,...,NaN,NaN,1.000,2.000,"1,731,783,600,000.000","1,731,790,800,000.000","Nov 16, 2:00PM",Open House - 2:00 - 4:00 PM,NaN,NaN
74681956,False,Active,True,False,True,3,2.500,2.000,Upper St. Clair,PA,...,NaN,NaN,1.000,NaN,"1,731,772,800,000.000","1,731,780,000,000.000","Nov 16, 11:00AM",Open House - 11:00 - 1:00 PM,NaN,NaN
